<a href="https://colab.research.google.com/github/dustinak80/Deep_Learning/blob/master/HW3/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 3 - Stock price prediction

---



## Import packages

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import Constant



## Import the Data

In [13]:
# show nice table in colab
%load_ext google.colab.data_table 
from google.colab import data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [15]:
stocks = pd.read_csv('https://raw.githubusercontent.com/dustinak80/Deep_Learning/master/HW3/stocks_dataset.csv')
data_table.DataTable(stocks, include_index=False, num_rows_per_page=10)

,Date,ABT,ABBV,ABMD,A,ALXN,ALGN,AGN,ABC,AMGN,ANTM,BAX,BDX,BIIB,BSX,BMY,CAH,CNC,CERN,CI,COO,CVS,DHR,DVA,XRAY,EW,GILD,HCA,HSIC,HOLX,HUM,IDXX,ILMN,INCY,ISRG,IQV,JNJ,LH,LLY,MCK,MDT,MRK,MTD,MYL,PKI,PRGO,PFE,DGX,REGN,RMD,STE
0,2015-01-02 00:00:00,44.900002,65.889999,37.310001,40.560001,186.600006,56.200001,259.130005,90.459999,159.889999,125.459999,39.820751,139.860001,342.010010,13.220000,59.509998,80.510002,26.110001,65.029999,102.830002,160.610001,95.099998,64.988625,75.830002,51.930000,63.860001,94.910004,73.970001,53.662746,26.379999,142.990005,73.665001,185.419998,73.760002,175.190002,59.110001,104.519997,111.419998,70.059998,207.199997,71.879997,57.189999,301.950012,56.349998,44.090000,166.630005,31.330000,68.970001,410.160004,57.169998,65.160004
1,2015-01-05 00:00:00,44.910000,64.650002,37.070000,39.799999,182.169998,56.990002,256.690002,89.690002,157.990005,123.260002,38.832157,140.720001,336.739990,13.810000,58.580002,80.050003,26.040001,64.169998,101.059998,159.369995,94.160004,64.344200,74.699997,51.570000,63.889999,96.790001,71.809998,53.180393,26.120001,139.199997,73.154999,190.720001,74.220001,171.456665,58.410000,103.790001,112.389999,69.370003,208.899994,71.580002,58.040001,295.450012,55.720001,43.049999,164.910004,31.160000,67.279999,412.470001,57.060001,64.510002
2,2015-01-06 00:00:00,44.400002,64.330002,36.130001,39.180000,177.949997,57.450001,255.169998,90.180000,152.899994,124.940002,38.625748,140.449997,334.649994,13.700000,58.480000,79.190002,26.125000,63.520000,101.459999,157.119995,94.230003,63.904472,73.620003,50.930000,63.509998,97.650002,71.690002,52.870586,25.750000,139.089996,73.389999,188.309998,71.489998,173.263336,57.070000,103.279999,110.599999,69.720001,208.419998,71.220001,60.320000,291.119995,54.180000,42.660000,164.600006,31.420000,66.269997,396.890015,56.650002,64.550003
3,2015-01-07 00:00:00,44.759998,66.930000,37.279999,39.700001,187.929993,59.570000,265.230011,91.980003,158.240005,125.559998,38.810429,143.740005,353.239990,14.030000,59.240002,82.029999,26.267500,64.190002,103.839996,162.220001,95.930000,64.291130,74.290001,52.250000,65.000000,99.480003,72.989998,54.333332,26.400000,141.729996,74.870003,194.919998,74.449997,174.213333,57.770000,105.559998,112.540001,69.230003,214.750000,73.000000,61.610001,296.489990,56.110001,42.919998,167.350006,31.850000,67.110001,407.720001,57.029999,65.070000
4,2015-01-08 00:00:00,45.680000,67.629997,38.959999,40.889999,183.800003,61.439999,268.959992,92.190002,157.669998,131.550003,39.815319,146.360001,350.250000,14.590000,60.720001,84.559998,26.672501,65.599998,109.129997,162.880005,97.970001,65.284309,75.769997,53.540001,66.574997,102.300003,74.930000,55.098038,27.110001,147.830002,75.415001,196.820007,73.209999,177.666672,59.200001,106.389999,115.570000,70.870003,217.199997,74.750000,62.849998,300.720001,56.650002,44.000000,171.169998,32.500000,68.879997,403.250000,57.990002,66.779999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,2020-04-20 00:00:00,98.000000,83.989998,166.199997,77.370003,106.720001,196.710007,187.389999,89.620003,236.600006,263.200012,94.139999,263.440002,339.410004,37.250000,61.680000,50.740002,68.419998,70.250000,188.699997,304.700012,62.340000,155.830002,77.519997,41.279999,224.600006,81.260002,110.430000,53.150002,43.240002,367.329987,267.339996,322.880005,101.190002,518.330017,127.510002,151.669998,145.610001,157.789993,140.490005,101.540001,83.099998,707.630005,15.860000,84.519997,50.860001,36.080002,95.029999,567.989990,164.279999,157.449997
1333,2020-04-21 00:00:00,94.050003,80.360001,164.500000,73.750000,103.389999,187.259995,184.039993,85.570000,230.750000,249.369995,90.099998,256.500000,329.010010,35.049999,60.590000,48.270000,66.239998,68.589996,177.910004,288.529999,60.250000,153.750000,72.529999,39.660000,216.020004,78.529999,105.470001,52.029999,42.349

## Data Cleaning

In [16]:
#any nulls?
print('There are {} null values\n'.format(stocks.isnull().sum().sum()))

There are 0 null values



### Time variable

In [22]:
#what are the dtypes of the columns
stocks.dtypes.value_counts()

float64    50
object      1
dtype: int64

Dates are not in date format. Lets get into date format.

In [23]:
stocks['Date'] = pd.to_datetime(stocks['Date'])
stocks['Date'].dtypes

dtype('<M8[ns]')

In [24]:
print('The shape is {}'.format(stocks.shape))

The shape is (1337, 51)


## Series to Supervised

In [0]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
  """
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""#n_vars = 1 if type(data) is list else data.shape[1]
  variables = list(data.columns)
  df = data.copy(deep = True)
  cols, names = list(), list()
  # input sequence (t-n, ... t-1)
  for i in range(n_in, 0, -1):
    cols.append(df.shift(i))
    names += ['{}(t-{})'.format(j, i) for j in variables]
  # forecast sequence (t, t+1, ... t+n)
  for i in range(0, n_out):
    cols.append(df.shift(-i))
    if i == 0:
      names += ['{}(t)'.format(j) for j in variables]
    else:
      names += ['{}(t+{})'.format(j, i) for j in variables]
  # put it all together
  agg = pd.concat(cols, axis=1)
  agg.columns = names
  # drop rows with NaN values
  if dropnan:
    agg.dropna(inplace=True)
  return agg

In [0]:
df = series_to_supervised(stocks.loc[:,'ABT':], n_in=5, n_out=2, dropnan=False)

In [0]:
print('new shape is {}'.format(df.shape))
#shape is 350 because it is all 50 stocks of t+1. Remove all except ABT
columns = list(df.columns)[-49:]

new shape is (1337, 350)


In [0]:
df.drop(columns = list(df.columns)[-49:], inplace = True)

In [0]:
df.shape

(1337, 301)